<a href="https://colab.research.google.com/github/Henry-0810/Artificial-Intelligence/blob/main/portfolio_optimization_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Genetic Algorithm**
## **Portfolio Optimization**
I will only use 5 stocks for this project. Stock data are all real-time, retrived from the Yahoo Finance API, https://pypi.org/project/yfinance/

## Problem Statement (~30%)

### Description of the Problem  
Portfolio optimization is the process of selecting the best combination of assets to maximize returns while minimizing risk. A common approach is **Markowitz’s Mean-Variance Optimization (MVO)**, which constructs an efficient portfolio using expected returns, variances, and covariances of assets.  

However, MVO has several limitations:  
1. **Complexity**: Solving quadratic optimization problems becomes expensive as the number of assets grows.  
2. **Strict Assumptions**: MVO assumes asset returns follow a normal distribution and that investors have quadratic utility functions, which is not always realistic.  
3. **Handling Constraints**: Incorporating real-world constraints (e.g., restricting allocations to certain stocks) increases the complexity.  

To overcome these challenges, this project applies a **Genetic Algorithm (GA)** to optimize portfolio allocation. Unlike MVO, GA is a heuristic approach that efficiently searches for near-optimal solutions without relying on strict mathematical assumptions.  

The goal is to maximize the **Sharpe Ratio**, a key financial metric that balances return and risk, making GA a suitable technique for solving this problem.

---

### Discussion of the Suitability of Genetic Algorithms
Genetic Algorithms (GAs) are well-suited for portfolio optimization due to their ability to handle large search spaces and complex constraints. Key advantages include:  

- **Scalability** – GAs can handle large datasets efficiently without exponential growth in computational cost.  
- **No Assumption of Normality** – Unlike MVO, GA does not require asset returns to be normally distributed.  
- **Flexibility** – GA can incorporate additional constraints, such as limiting investment in specific stocks or ensuring minimum allocation to certain sectors.  

Through **evolutionary selection**, GA gradually improves the portfolio allocation, leading to **higher Sharpe Ratios** compared to naive allocation strategies.

---

### Complexity of the Problem (Project Plan)
Portfolio optimization presents a **high-dimensional, non-linear problem** where:  
- Each chromosome represents a possible portfolio allocation (weights og assets).  
- The solution space grows exponentially with the number of stocks considered.  
- Evaluating a portfolio requires calculating historical **returns, variances, and covariances** over a defined period.  
- The GA must balance exploration (mutation) and exploitation (crossover) to meet an optimal solution efficiently.  

The complexity of the Genetic Algorithm increases with:  
- The **number of assets** included in the portfolio (will be using only 5 in this case for a proof of concept).  
- The **historical data period** used for return calculations (longer periods increase computational load).  
- The **GA parameters** (population size, mutation rate, number of generations), which impact convergence speed.  

Despite this complexity, GA provides a robust and adaptable approach to solving portfolio optimization problems where traditional methods struggle.

---

**Reference**  
- [Markowitz Portfolio Theory](https://sites.math.washington.edu/~burke/crs/408/fin-proj/mark1.pdf)


## The Problem and Fitness Function (~20%)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from google.colab import drive

#### 1. Fetch Historical Data
I will fetch 6 months of historical data starting from August 11th 2024 to February 11th 2025.
I will investigate on 5 Stocks:
1. Apple Inc. (AAPL)
2. Tesla Inc. (TSLA)
3. Microsoft Corp. (MSFT)
4. Amazon.com Inc. (AMZN)
5. NVIDIA Corporation (NVDA)

In [2]:
start_date = "2024-08-11"
end_date = "2025-02-11"

stocks = ["AAPL","TSLA","MSFT","AMZN","NVDA"]
data = yf.download(stocks, start=start_date, end=end_date, interval="1d")
print(data.columns)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed

MultiIndex([( 'Close', 'AAPL'),
            ( 'Close', 'AMZN'),
            ( 'Close', 'MSFT'),
            ( 'Close', 'NVDA'),
            ( 'Close', 'TSLA'),
            (  'High', 'AAPL'),
            (  'High', 'AMZN'),
            (  'High', 'MSFT'),
            (  'High', 'NVDA'),
            (  'High', 'TSLA'),
            (   'Low', 'AAPL'),
            (   'Low', 'AMZN'),
            (   'Low', 'MSFT'),
            (   'Low', 'NVDA'),
            (   'Low', 'TSLA'),
            (  'Open', 'AAPL'),
            (  'Open', 'AMZN'),
            (  'Open', 'MSFT'),
            (  'Open', 'NVDA'),
            (  'Open', 'TSLA'),
            ('Volume', 'AAPL'),
            ('Volume', 'AMZN'),
            ('Volume', 'MSFT'),
            ('Volume', 'NVDA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])


There are 5 types of Prices: Close, High, Low, Open, Volume. Close prices are often used for analysis because it reflects the final consensus on the stock price for the trading day. It is often most consistent for portfolio optimization.

In [3]:
close_data = data["Close"]
drive.mount('/content/drive')

close_data.to_csv('/content/drive/My Drive/stock_data.csv')

Mounted at /content/drive


In [4]:
stock_data = pd.read_csv('/content/drive/My Drive/stock_data.csv', index_col=0, parse_dates=True)
stock_data.head(10)

,AAPL,AMZN,MSFT,NVDA,TSLA
Date,,,,,
2024-08-12,217.052292,166.800003,404.455902,108.993134,197.490005
2024-08-13,220.784073,170.229996,411.614227,116.111382,207.830002
2024-08-14,221.233093,170.100006,414.447723,118.050911,201.380005
2024-08-15,224.226501,177.589996,419.348083,122.829727,214.139999
2024-08-16,225.553589,177.059998,416.798309,124.549309,216.119995
2024-08-19,225.393936,178.220001,419.846069,129.967957,222.720001
2024-08-20,226.012573,178.880005,423.103027,127.218651,221.100006
2024-08-21,225.902802,180.110001,422.445679,128.468353,223.270004
2024-08-22,224.036926,176.130005,413.889954,123.709503,210.660004


#### 2. Preprocess Data (Compute Returns and Risk)

Before defining the fitness function **(Sharpe Ratio)**, I need to compute some key financial metrics from the historical stock data.

Portfolio optimization requires **2** fundamental components:
1. Returns → Measures how an asset appreciates over time
2. Risk (Standard Deviation & Covariance) → Measures relationship between assets

To Calculate returns and risk, **3** statistical methods are used:
- Daily Returns: estimate expected return of portfolio
- Covariance Matrix: measure risk relationship between different stocks
- Standard Deviation: quantify portfolio risk

Without calculating these, Sharpe Ratio cannot be correctly evaluated.

In [5]:
daily_returns = stock_data.pct_change().dropna()

expected_returns = daily_returns.mean()

covariance_matrix = daily_returns.cov()

print("Daily Return:\n",daily_returns)
print("\nExpected Return:\n", expected_returns)
print("\nCovariance Matrix:\n", covariance_matrix)

Daily Return:
                 AAPL      AMZN      MSFT      NVDA      TSLA
Date                                                        
2024-08-13  0.017193  0.020564  0.017699  0.065309  0.052357
2024-08-14  0.002034 -0.000764  0.006884  0.016704 -0.031035
2024-08-15  0.013531  0.044033  0.011824  0.040481  0.063363
2024-08-16  0.005919 -0.002984 -0.006080  0.014000  0.009246
2024-08-19 -0.000708  0.006551  0.007312  0.043506  0.030539
...              ...       ...       ...       ...       ...
2025-02-04  0.021008  0.019543  0.003529  0.017058  0.022232
2025-02-05 -0.001418 -0.024333  0.002231  0.052086 -0.035797
2025-02-06  0.003226  0.011263  0.006122  0.030842 -0.010181
2025-02-07 -0.023969 -0.040531 -0.014598  0.009015 -0.033928
2025-02-10  0.001187  0.017412  0.006028  0.028728 -0.030114

[124 rows x 5 columns]

Expected Return:
 AAPL    0.000475
AMZN    0.002860
MSFT    0.000236
NVDA    0.002209
TSLA    0.005521
dtype: float64

Covariance Matrix:
           AAPL      AMZN    

#### **Observation**

Expected Returns:
- TSLA (0.005521) has the highest expected return, indicating strong growth potential but also higher risk.
- AMZN (0.002860) and NVDA (0.002209) follow, suggesting moderate expected returns.
- AAPL (0.000475) and MSFT (0.000236) have the lowest expected returns, implying more stability.

Covariance Matrix:
- TSLA and NVDA show the highest variances, indicating greater price fluctuations.
- AMZN is moderately correlated with NVDA and TSLA, potentially indicating similar market influences.
- AAPL and MSFT have lower variances, reinforcing their status as more stable assets.
---
**Reference**:
- Markowitz, H. (1952). *Portfolio Selection.* The Journal of Finance, 7(1), 77-91. DOI: [10.2307/2975974](https://doi.org/10.2307/2975974)

#### 3. Sharpe Ratio - Fitness Function
**Introduction of Sharpe Ratio** <br>
The fitness function in my Genetic Algorithm is the **Sharpe Ratio**, a key financial metric that measures risk-adjusted return. <br>
Formulas below:
\begin{aligned}
S &= \frac{E(R_p) - R_f}{\sigma_p}
\end{aligned}

where:  
- $S$ = Sharpe Ratio  
- $E(R_p)$ = Expected portfolio return  
- $R_f$ = Risk-free rate (Since the portfolio consists only 5 stocks and does not include a risk-free asset like government bonds, assuming 0 simplifies calculations without affecting the optimization objective.)  
- $\sigma_p$ = Portfolio standard deviation (risk)  

\begin{aligned}
E(R_p) = \sum_{i=1}^{N} w_i E(R_i)
\end{aligned}

where:
- $w_i$ = weight of stock $i$
- $R_i$ = return of stock $i$


The Sharpe Ratio is widely used in finance because it provides a balanced measure of risk vs. return. Unlike simple return-based metrics, it penalizes high volatility, ensuring that we optimize portfolios that offer steady and reliable returns.

---

**Why use Sharpe Ratio as a Fitness Function?**
1. Risk-Adjusted Performance: Ensures GA selects portfolios that maximize return while controlling risk.  
2. Industry Standard Metric: Used by hedge funds and investment firms for portfolio evaluation.  
3. Handles Different Market Conditions: Works well in both volatile and stable markets.  

By using Genetic Algorithms, we evolve portfolios that maximize the Sharpe Ratio, leading to an optimal allocation of assets that achieves the best trade-off between *profitability and risk*.

---

**Reference:**  
- Sharpe, W. F. (1966). "Mutual Fund Performance". *Journal of Business*, 39(1), 119-138. ([JSTOR](https://www.jstor.org/stable/2351741))  

In [6]:
def sharpe_ratio(weights, expected_returns, covariance_matrix, risk_free_rate=0.0):
  """
  Parameters:
  - weights: Array of portfolio allocation (sum to 1)
  - expected_returns: Expected return of each stocks
  - covariance_matrix: Covariance matrix of stock returns
  - risk_free_rates: default = 0

  Returns:
  - Sharpe Ratio (higher = better/optimized)
  """

  # Reference to Sharpe Ratio formulas above
  portfolio_return = np.dot(weights, expected_returns)

  # Transpose weights to get a single variance value
  portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
  portfolio_stddev = np.sqrt(portfolio_variance)

  return (portfolio_return - risk_free_rate) / portfolio_stddev

Testing Fitness Function

In [7]:
test_expected_return = np.array([0.12, 0.08, 0.10])
test_covariance_matrix = np.array([
    [0.04, 0.01, 0.02],
    [0.01, 0.03, 0.015],
    [0.02, 0.015, 0.05]
])

test_weights = np.array([0.4, 0.3, 0.4])

sharpe_value = sharpe_ratio(test_weights, test_expected_return, test_covariance_matrix)

print("Sharpe Ratio", sharpe_value)

Sharpe Ratio 0.6520892109083318


### Problem Class

In [8]:
class Problem:
  def __init__(self) -> None:
    self.number_of_genes = 5
    self.min_value = 0
    self.max_value = 1
    self.fitness_function = sharpe_ratio
    self.acceptable_fitness = 1 # < 1 = weak (but show stability), 1 - 1.5 = performing alright, > 1.5 = strong (at the same time risky)

## The Individual (~30%) - Defining the Genetic Algorithm Components

### Individual Components
#### Chromosome Representation
In this project, each chromosome represents a portfolio allocation, where genes are the weights of individual stocks in the portfolio. Each chromosome is an array of floating-point numbers summing up to 1.

For example: a chromosome might look like:
*\[0.25, 0.15, 0.30, 0.10, 0.20\]*

Why this representation? <br>
- Ensures a valid portfolio allocation without needing extra constraints.  
- Allows easy evaluation using the Sharpe Ratio as the fitness function.  
- Works naturally with crossover and mutation without breaking feasibility.  


These weights involve over generations to maximize the Sharpe Ratio.

---

#### Crossover (Recombination of Portfolios)
Crossover allows 2 parent portfolios to combine, creating a new portfolio that inherits some traits from both.

**Approach**
- Linear Interpolation Crossover: This method ensures that the child portfolios always sum to 1, maintaining valid asset allocation without perfoming normalization.

Justification of crossover approach: <br>
- Always maintains valid weights (sum = 1) without extra renormalization.  
- Encourages genetic diversity by allowing more flexible exploration.  
- Works better for constrained problems like portfolio allocation compared to traditional crossover (e.g., BLX crossover, which requires post-normalization).  

---

#### Mutation
Mutation helps prevent the GA from converging too quickly to a local optimum. We apply small random adjustments to portfolio weights.

**Approach**: Small Weight Perturbation with Renormalization
1. A small noise is added to randomly selected weights.  
2. Weights are clipped to prevent negative values (ensuring no short-selling).  
3. The portfolio is renormalized to maintain a total weight sum of 1.

Justification of mutation method: <br>
- Ensures that the weights always sum to 1 after mutation.  
- Prevents negative weights, ensuring all assets have a valid allocation.  
- Controlled noise to adjust weights without drastic changes.

This ensures mutation improves the solution without breaking constraints.

In [15]:
from copy import deepcopy

class PortfolioIndividual:
  def __init__(self, prob):
    """
    Initializes an individual portfolio (chromosomes)

    Parameters:
    - prob: An Instance of the Problem class
    """
    self.chromosomes = np.random.dirichlet(np.ones(prob.number_of_genes), size=1)[0]
    self.fitness = prob.fitness_function

  def crossover(self, parent2, explore_crossover=0.1):
    """
    Linear Interpolation Crossover, more details above

    Parameters:
    - parent2: Another PortfolioIndividual to crossover with
    - explore_crossover: Exploration factor (default 0.1)

    Returns:
    - 2 children PortfolioIndividuals
    """
    alpha = np.random.uniform(-explore_crossover, 1 + explore_crossover)

    child1 = deepcopy(self)
    child2 = deepcopy(parent2)

    child1.chromosomes = alpha * self.chromosomes + (1 - alpha) * parent2.chromosomes
    child2.chromosomes = alpha * parent2.chromosomes + (1 - alpha) * self.chromosomes

    return child1, child2

  def mutate(self, mutation_rate=0.1, mutation_strength=0.05):
    """
    Applies mutation by slightly adding noise to random portfolio weights

    Parameters:
    - mutation_rate: Probability of mutation to happen for each weight
    - mutation_strength: Standard deviation of noise added

    Returns:
    - Mutated PortfolioIndividual
    """
    mutated_portfolio = deepcopy(self)

    for i in range(len(mutated_portfolio.chromosomes)):
      if np.random.rand() < mutation_rate:
        mutated_portfolio.chromosomes[i] += np.random.normal(0, mutation_strength)

    mutated_portfolio.chromosomes = np.clip(mutated_portfolio.chromosomes, 0, 1)

    mutated_portfolio.chromosomes /= np.sum(mutated_portfolio.chromosomes)

    return mutated_portfolio



## Running the GA algorithm (~10%)


## Selection, Parameters and GA
### Parameter Choices
To optimize portfolio allocation, we define the following Genetic Algorithm parameters:  

- Population Size: 50 portfolios per generation balance between diversity & efficiency.  
- Number of Generations: 100 iterations to ensures sufficient evolution.  
- Mutation Rate: 10% introduces variation without excessive randomness.  
- Mutation Strength: 5% so noise is controlled.  
- Crossover Rate: 80% to ensure effective recombination.  

These parameters were chosen based on standard GA heuristics and financial constraints.

---

### Selection Method: Tournament Selection
To ensure only the best-performing portfolios move to the next generation, I plan to use **Tournament Selection**:  

How it Works?
- Randomly pick 3 portfolios from the population.  
- The portfolio with the **highest Sharpe Ratio** wins.  
- Repeat until enough parents are selected.

Tournament Selection vs. Roulette Wheel:
- More stable because it avoids overly favoring extreme values.  
- Better suited for small population sizes like this project.

Reference below...

---

### Modifications to `run_genetic`
1. Implemented Tournament Selection for better stability.  
2. Dynamically adjusted mutation rate to avoid premature convergence.  
3. Tracked convergence metrics to assess solution quality.

---

#### **Reference**
- [Tournament Selection](https://www.baeldung.com/cs/ga-tournament-selection)
- [Comparison between tournament and roulette wheel](https://www.researchgate.net/publication/221216912_Comparison_of_Performance_between_Different_Selection_Strategies_on_Simple_Genetic_Algorithms)

In [16]:
class Parameters:
  def __init__(self):
    self.population_size = 50
    self.number_of_generations = 100
    self.mutation_rate = 0.1
    self.mutation_strength = 0.05
    self.explore_crossover_range = 0.2
    self.birth_rate_per_generation = 1

In [26]:
def choose_parents(population, tournament_size=3):
  """
  Tournament Selection: choose 3 random parents, calculate their sharpe ratio and return the highest.

  Parameters:
  - tournament_size: number of contestents (i.e. number of parents chosen to compete in this case)

  Returns:
  - 2 parents from 2 tournaments with highest fitness function (Sharpe Ratio)
  """
  tournament1 = np.random.choice(population, tournament_size)
  tournament2 = np.random.choice(population, tournament_size)

  parent1 = max(tournament1, key=lambda x: x.fitness(x.chromosomes, expected_returns, covariance_matrix))
  parent2 = max(tournament2, key=lambda x: x.fitness(x.chromosomes, expected_returns, covariance_matrix))

  return parent1, parent2


In [27]:
def run_genetic(prob, params):
  # 1. Read variables
  num_of_population = params.population_size
  rate_of_gene_mutation = params.mutation_rate
  mutation_strength = params.mutation_strength
  explore_crossover = params.explore_crossover_range
  max_num_of_generations = params.number_of_generations

  fitness_function = prob.fitness_function
  acceptable_fitness = prob.acceptable_fitness
  num_of_child_per_generation = num_of_population * params.birth_rate_per_generation

  # 2. Create population
  population = []
  best_solution = PortfolioIndividual(prob)
  best_solution.fitness = -np.inf

  for i in range(num_of_population):
    new_individual = PortfolioIndividual(prob)
    new_individual.fitness = fitness_function(new_individual.chromosomes, expected_returns, covariance_matrix)

    if new_individual.fitness > best_solution.fitness:
      best_solution = deepcopy(new_individual)

    population.append(new_individual)

  print(f"Initial Population Size: {len(population)}")

  # 3. Start GA Evolution Loop
  for gen in range(max_num_of_generations):
    children = []

    # Generate children through crossover and mutation
    while(len(children) < num_of_child_per_generation):
      parent1, parent2 = choose_parents(population=population)

      child1, child2 = parent1.crossover(parent2, explore_crossover)

      child1 = child1.mutate(rate_of_gene_mutation, mutation_strength)
      child2 = child2.mutate(rate_of_gene_mutation, mutation_strength)

      child1.fitness = fitness_function(child1.chromosomes, expected_returns, covariance_matrix)
      child2.fitness = fitness_function(child2.chromosomes, expected_returns, covariance_matrix)

      children.append(child1)
      children.append(child2)

    # 4. Add children to new population
    population += children

    # 5. Sort
    population = sorted(population, key=lambda x: x.fitness_function, reverse=True)

    # 6. Cull population to fixed size
    population = population[:num_of_population]

    if population[0].fitness > best_solution.fitness:
      best_solution = deepcopy(population[0])
      print(f"Generation {gen + 1}: New Best Sharpe Ratio = {best_solution.fitness}")

    if best_solution.fitness > acceptable_fitness:
      print("Optimal portfolio found, stopping early.")
      break

  return population, best_solution

Testing on GA

In [28]:
p = Problem()

In [29]:
assets = PortfolioIndividual(p)
print(assets.chromosomes)
print(sum(assets.chromosomes))

[0.14801684 0.41587555 0.09482339 0.22435967 0.11692455]
1.0


In [30]:
params = Parameters()

In [31]:
population, best_solution = run_genetic(p, params=params)
population, best_solution.chromosomes

Initial Population Size: 50


TypeError: 'numpy.float64' object is not callable